In [1]:
import random
import networkx as nx
import plotly.graph_objects as go
from IPython.display import display, clear_output
import time
import re
import json
import hashlib
import os
from ollama import chat
import ipywidgets as widgets
from ipywidgets import VBox, Button


In [ ]:

# --------------------------
# Thought Node and Edge Classes
# --------------------------
class ThoughtNode:
    def __init__(self, phrase, awareness, contradiction, recursion, intensity, concept="unknown", state='superposed'):
        self.phrase = phrase
        self.awareness = awareness
        self.contradiction = contradiction
        self.recursion = recursion
        self.intensity = intensity
        self.concept = concept
        self.state = state
        self.links = []
        self.position = (0, 0, 0)  # ← new field to store coordinates

class ThoughtEdge:
    def __init__(self, source, target, weight=1.0, type='logical'):
        self.source = source
        self.target = target
        self.weight = weight
        self.type = type # not used yet


In [3]:

# --------------------------
# Cache Setup for Reproducibility
# --------------------------
CACHE_FILE = "thought_cache.json"
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        THOUGHT_CACHE = json.load(f)
else:
    THOUGHT_CACHE = {}

def save_cache():
    with open(CACHE_FILE, "w") as f:
        json.dump(THOUGHT_CACHE, f, indent=2)

def clear_cache():
    global THOUGHT_CACHE
    THOUGHT_CACHE = {}
    save_cache()
    print("Cache cleared.")

def hash_thought(phrase: str) -> str:
    normalized = phrase.strip().lower()
    return hashlib.md5(normalized.encode('utf-8')).hexdigest()


In [4]:

# --------------------------
# Auto-Rating with Ollama (Deterministic + Caching)
# --------------------------
def rate_thought_nlp(phrase):
    phrase = phrase.strip().lower()
    key = hash_thought(phrase)
    if key in THOUGHT_CACHE:
        return tuple(THOUGHT_CACHE[key])

    prompt = (
        f"Evaluate the following thought by analyzing its cognitive structure. "
        f"Consider how aware or insightful it is (awareness), how much internal conflict or contradiction it contains, "
        f"whether it reflects on itself or involves nested reasoning (recursion), and how intense or emotionally charged it feels (intensity). "
        f"Then assign a one-word concept that best captures its essence.\n\n"
        f"Thought: \"{phrase}\"\n\n"
        f"Respond ONLY with a JSON like this:\n"
        f"{{\n"
        f"  \"awareness\": <0-1>,\n"
        f"  \"contradiction\": <0-1>,\n"
        f"  \"recursion\": <0-5>,\n"
        f"  \"intensity\": <0-1>,\n"
        f"  \"concept\": \"<one-word>\"\n"
        f"}}"
    )

    try:
        response = chat(
            model='llama3.2',
            messages=[
                {'role': 'system', 'content': 'Respond only with a valid JSON dictionary. No explanations.'},
                {'role': 'user', 'content': prompt}
            ],
            options={"temperature": 0}
        )

        response_text = response['message']['content']
        print("Raw response:", response_text)  # debug print
        output = json.loads(response_text.strip())
        result = (
            float(output["awareness"]),
            float(output["contradiction"]),
            int(output["recursion"]),
            float(output["intensity"]),
            output["concept"]
        )
        THOUGHT_CACHE[key] = result
        save_cache()
        return result
    except Exception as e:
        print("Error using Ollama chat API:", e)
        return (0.5, 0.5, 1, 0.5, "unknown")


In [5]:
# --------------------------
# Thought Space Initialization (temporary random weights)
# --------------------------
def create_thought_space_from_phrases(phrases):
    thoughts = []

    # Create nodes
    for phrase in phrases:
        phrase = phrase.strip().lower()
        awareness, contradiction, recursion, intensity, concept = rate_thought_nlp(phrase)
        t = ThoughtNode(
            phrase=phrase,
            awareness=awareness,
            contradiction=contradiction,
            recursion=recursion,
            intensity=intensity,
            concept=concept
        )
        thoughts.append(t)

    # Create edges
    edges = []
    for i in range(len(thoughts)):
        for j in range(i+1, len(thoughts)):
            prompt = (
                f"Should the following two thoughts be connected logically?\n\n"
                f"Thought 1: \"{thoughts[i].phrase}\"\n"
                f"Thought 2: \"{thoughts[j].phrase}\"\n\n"
                f"Respond ONLY with 'yes' or 'no'."
            )
            try:
                stream = chat(
                    model='llama3.2',
                    messages=[
                        {'role': 'system', 'content': 'Respond only with yes or no. No explanation.'},
                        {'role': 'user', 'content': prompt}
                    ],
                    stream=True,
                    options={"temperature": 0}
                )
                response_text = ""
                for chunk in stream:
                    response_text += chunk['message']['content']
                
                if 'yes' in response_text.lower():
                    # Random weight for now
                    edge = ThoughtEdge(thoughts[i], thoughts[j], weight=random.random())
                    thoughts[i].links.append(thoughts[j])
                    thoughts[j].links.append(thoughts[i])
                    edges.append(edge)
            except Exception as e:
                print(f"Error evaluating edge between \"{thoughts[i].phrase}\" and \"{thoughts[j].phrase}\":", e)

    return thoughts, edges


Formula explained:

Each thought can influence connected thoughts in the network.

The amount of influence is a small fraction (5%) of the source thought’s intensity.

This models ideas propagating through a thought network: strong thoughts make related thoughts more “active” or noticeable.

Using min(neighbor.intensity, 1.0) keeps intensity in a valid range (0 to 1).

In [9]:

# --------------------------
# Update Logic
# --------------------------

def update_thoughts(thoughts):
    for t in thoughts:
        # Collapse superposed thoughts based on awareness
        if t.state == 'superposed' and t.awareness > 0.7:
            t.state = 'collapsed'

        # Spread intensity to neighbors (simplified)
        for neighbor in t.links:
            delta = t.intensity * 0.05  # purely based on source intensity
            neighbor.intensity += delta
            neighbor.intensity = min(neighbor.intensity, 1.0)

In [7]:

# --------------------------
# 3D Visualization
# --------------------------
def visualize_3d(thoughts, edges):
    G = nx.Graph()
    for idx, t in enumerate(thoughts):
        G.add_node(idx)
    for e in edges:
        G.add_edge(thoughts.index(e.source), thoughts.index(e.target))

    pos = nx.spring_layout(G, dim=3, seed=42)
    for n in G.nodes:
        thoughts[n].position = pos[n]  # Store position in node

    node_x = [pos[n][0] for n in G.nodes]
    node_y = [pos[n][1] for n in G.nodes]
    node_z = [pos[n][2] for n in G.nodes]

    concept_colors = {}
    available_colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'brown']

    def get_concept_color(concept):
        if concept not in concept_colors:
            concept_colors[concept] = available_colors[len(concept_colors) % len(available_colors)]
        return concept_colors[concept]

    node_color = [get_concept_color(t.concept) for t in thoughts]

    edge_x = []
    edge_y = []
    edge_z = []
    for e in G.edges:
        x0, y0, z0 = pos[e[0]]
        x1, y1, z1 = pos[e[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
        edge_z += [z0, z1, None]

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=1, color='gray'),
        hoverinfo='none',
        mode='lines'
    )

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(
            size=8,
            color=node_color,
            opacity=0.8
        ),
        text=[f"Phrase: {t.phrase}<br>Awareness: {t.awareness:.2f}<br>Contradiction: {t.contradiction:.2f}<br>Recursion: {t.recursion}<br>Intensity: {t.intensity:.2f}" for t in thoughts],
        hoverinfo='text'
    )

    fig = go.Figure(data=[edge_trace, node_trace])
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    clear_output(wait=True)
    display(fig)

    print("\n--- Thought Positions ---")
    for t in thoughts:
        print(f"{t.phrase} → {t.position}")


In [8]:

# --------------------------
# Static Input via Variable
# --------------------------
phrases = """
I think, therefore I am.
Can a machine become conscious of its own thoughts?
The universe is an illusion experienced by consciousness.
To know that you know nothing is the beginning of wisdom.
Does free will exist, or is everything predetermined?
A lie that is believed becomes truth in practice.
If everything is connected, can anything be truly independent?
What observes the observer?
"""

if __name__ == '__main__':
    clear_cache_button = Button(description="Clear Cache", button_style='danger')
    clear_cache_button.on_click(lambda x: clear_cache())
    display(clear_cache_button)

    phrase_list = [line.strip() for line in phrases.strip().split('\n') if line.strip()]
    thoughts_global, edges_global = create_thought_space_from_phrases(phrase_list)
    update_thoughts(thoughts_global)
    visualize_3d(thoughts_global, edges_global)


--- Thought Positions ---
i think, therefore i am. → [0.41148314 1.         0.58616001]
can a machine become conscious of its own thoughts? → [-0.08637327  0.09011764 -0.6283866 ]
the universe is an illusion experienced by consciousness. → [-0.12447294 -0.1062233  -0.16160936]
to know that you know nothing is the beginning of wisdom. → [ 0.11546298 -0.47279091  0.14133923]
does free will exist, or is everything predetermined? → [ 0.45215266 -0.13704948 -0.36165267]
a lie that is believed becomes truth in practice. → [-0.3763507  -0.26577193  0.44435496]
if everything is connected, can anything be truly independent? → [0.07391788 0.20716338 0.11814361]
what observes the observer? → [-0.46581974 -0.3154454  -0.13834917]
